<a href="https://colab.research.google.com/github/richirey75/CS2450-Final-Project/blob/main/DataMiningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Here I am mounting my drive and open a new folder to place all the content of the project

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# !mkdir /content/drive/MyDrive/DataMiningProject/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## To make shell commands we need the "!" before the command

In [ ]:
!python --version

Python 3.10.12


The % is a special command in Google Colab and Jupyter notebooks called a "magic command". These commands provide special functionalities.

%cd is one such magic command which is used to change the current working directory.

In [ ]:
%cd /content/drive/MyDrive/DataMiningProject
!pwd

/content/drive/MyDrive/DataMiningProject
/content/drive/MyDrive/DataMiningProject


## USEFUL SHORTSCUTS

- Ctrl + M + H: Show all shortcuts
- Ctrl + M + B: Create a new cell below
- Ctrl + M + A: Create a new cell above
- Ctrl + M + D: Delete the current cell
- Ctrl + M + Y: Change cell to code
- Ctrl + M + M: Change cell to text
- Ctrl + Enter: Run the current cell
- Ctrl + Shift + Enter: Run the current cell and select the below cell

In [ ]:
!pip install spotipy --upgrade

## We need to import our libraries we will be using for the project

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

## This is how we will identify to the API

In [ ]:
client_id = input("Enter your client_id: ")
client_secret = input("Enter your client_secret: ")

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

## I have use the search function to query an artist

In [ ]:
results = sp.search(q='The Cranberries', limit=1, type='artist')
artist_id = results['artists']['items'][0]['id']
artist_id

'7t0rwkOPGlDPEhaOcVtOt9'

## Here is how to retreve albums from the artist_id from above

In [ ]:
albums = sp.artist_albums(artist_id, album_type='album')

album_names = [album['name'] for album in albums['items']]
album_names

['To The Faithful Departed (Deluxe Edition)',
 'Remembering Dolores',
 'No Need To Argue (Remastered 2020)',
 'No Need To Argue (Deluxe)',
 'In the End',
 'Something Else',
 'Live At the Hammersmith Apollo, London 2012',
 'Roses',
 'Live 2010 - Zenith Paris, 22.03.10',
 'Bualadh Bos: The Cranberries Live',
 'Wake Up And Smell The Coffee',
 'Bury The Hatchet (The Complete Sessions 1998-1999)',
 'Bury The Hatchet',
 'To The Faithful Departed (The Complete Sessions 1996-1997)',
 'To The Faithful Departed',
 'No Need To Argue (The Complete Sessions 1994-1995)',
 "Everybody Else Is Doing It, So Why Can't We? - Super Deluxe",
 "Everybody Else Is Doing It, So Why Can't We? (The Complete Sessions 1991-1993)",
 "Everybody Else Is Doing It, So Why Can't We?"]

## Here the code gives back the album songs from the [18] position on the list above

In [ ]:
tracks = sp.album_tracks(albums['items'][18]['id'])
track_names = [track['name'] for track in tracks['items']]
track_names

['I Still Do',
 'Dreams',
 'Sunday',
 'Pretty',
 'Waltzing Back',
 'Not Sorry',
 'Linger',
 'Wanted',
 "Still Can't...",
 'I Will Always',
 'How',
 'Put Me Down']

In [ ]:
df = pd.DataFrame({'Track Names': track_names})
df.to_csv('track_names.csv', index=False)

# This is the code to run to retrive the data from the artist you want


In [ ]:
# Artist information
artist_name = input("Enter an artist name: ")
results = sp.search(q=artist_name, limit=1, type='artist')
artist = results['artists']['items'][0]

artist_info = {
    'name': artist['name'],
    'id': artist['id'],
    'popularity': artist['popularity'],
    'genres': artist['genres'],
    'followers': artist['followers']['total']
}

# Album and track information with audio features
albums = sp.artist_albums(artist['id'], album_type='album')
all_tracks = []

for album in albums['items']:
    album_info = {
        'album_name': album['name'],
        'album_id': album['id'],
        'release_date': album['release_date'],
        'total_tracks': album['total_tracks']
    }

    tracks = sp.album_tracks(album['id'])
    for track in tracks['items']:
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'duration_min': track['duration_ms'] / 60000
        }
        try:
            track_info['popularity'] = track['popularity']
        except KeyError:
            track_info['popularity'] = None

        # Get audio features
        audio_features = sp.audio_features(track['id'])[0]
        if audio_features:
            track_info.update(audio_features)

        all_tracks.append({**album_info, **track_info})

# Create DataFrames
artist_df = pd.DataFrame([artist_info])
tracks_df = pd.DataFrame(all_tracks)

# Save to CSV
artist_df.to_csv(f'{artist_name}_artist_info.csv', index=False)
tracks_df.to_csv(f'{artist_name}_tracks_info.csv', index=False)

Enter an artist name: Bee Gees


## Here is to concatinate the SCV files

In [ ]:
df1 = pd.read_csv('Oasis_tracks_info.csv')
df2 = pd.read_csv('cranberries_tracks_info.csv')




combined_df = pd.concat([df1, df2]) # add more DataFrames to the list if needed
combined_df.to_csv('combined_tracks_info.csv', index=False)